# PySpark Example of Pyspark ML

Based on **Krish Naik** video: https://www.youtube.com/watch?v=_C8kWso4ne4

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ML').getOrCreate()
spark.sparkContext.setLogLevel("WARN")

In [4]:
# Read the dataset
training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [5]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

## VectorAssembler

### Concept:



``` 
[Age, Experience]----> new feature----> independent feature
```

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["age", "Experience"], outputCol="Independent Features")

In [9]:
output = featureassembler.transform(training)

In [10]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [12]:
finalized_data = output.select("Independent Features", "Salary")

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

23/06/28 07:26:53 WARN Instrumentation: [8122a880] regParam is zero, which might cause numerical instability and overfitting.
23/06/28 07:26:54 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/28 07:26:54 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [15]:
# Coefficients
regressor.coefficients

DenseVector([-250.0, 1750.0])

In [16]:
# Intercepts
regressor.intercept

20250.0000000005

In [17]:
# Prediction
pred_results = regressor.evaluate(test_data)

In [18]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000| 16750.00000000007|
|          [24.0,3.0]| 20000|19500.000000000036|
|          [30.0,8.0]| 25000|26749.999999999993|
+--------------------+------+------------------+



In [19]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1333.333333333342, 2125000.00000006)